In [ ]:
!apt-get install p7zip
!p7zip -d -f -k ../input/mercari-price-suggestion-challenge/train.tsv.7z
!unzip -o ../input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip -o ../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip
!p7zip -d -f -k ../input/mercari-price-suggestion-challenge/test.tsv.7z

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_csv('train.tsv', sep='\t').sample(frac=0.015).reset_index()

In [ ]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [ ]:
# !pip uninstall -y pyarrow
# !pip uninstall -y pyarrow
# !pip uninstall -y pyarrow

In [ ]:
# !rm -r ./autox
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

In [ ]:
from autox.autox_nlp import NLP_feature
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [ ]:
use_Toknizer=True
emb_mode = 'Bert'# TFIDF / Word2Vec / Glove / FastText / Bert
encode_mode = 'supervise' # unsupervise / supervise
text_columns_name = ['name','category_name','item_description']
target_column = df_train['price']
candidate_labels=None

In [ ]:
nlp = NLP_feature()
# nlp.do_mlm = True
# nlp.mlm_epochs=3
# nlp.model_name = 'microsoft/deberta-v3-base'
nlp.emb_size=100
nlp.n_clusters=20
train_meta = nlp.fit_transform(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)

In [ ]:
for column in train_meta.columns:
    df_train[column] = train_meta[column]
df_train = df_train.drop(columns=text_columns_name)

In [ ]:
test_meta = nlp.transform(df_test)

In [ ]:
for column in test_meta.columns:
    df_test[column] = test_meta[column]
df_test = df_test.drop(columns=text_columns_name)

In [ ]:
df_train.to_csv(f'{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
df_test.to_csv(f'{emb_mode}_{encode_mode}_autox_val.csv',index=False)

In [ ]:
df_val=pd.read_csv(f'{emb_mode}_{encode_mode}_autox_val.csv').drop(columns=['price'])
df_val.to_csv(f'{emb_mode}_{encode_mode}_autox_tst.csv',index=False)

In [ ]:
from autox import AutoX

path = f'.' 
autox = AutoX(target = 'price', train_name = f'{emb_mode}_{encode_mode}_autox_trn.csv', test_name = f'{emb_mode}_{encode_mode}_autox_tst.csv',  id = [], path = path)
sub = autox.get_submit()

In [ ]:
val = pd.read_csv(f'sub_val.csv')
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(val['price'], sub['price']))

In [ ]:
RMSE